In [71]:
import nltk
from google.colab import drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
nltk.download('punkt')
nltk.download('stopwords')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [72]:
import pandas as pd
data= pd.read_csv("/content/drive/MyDrive/CSE508_Winter2024_A2_2020559/A2_Data.csv")
data


,Unnamed: 0,Image,Review Text
0,3452,['https://images-na.ssl-images-amazon.com/imag...,Loving these vintage springs on my vintage str...
1,1205,['https://images-na.ssl-images-amazon.com/imag...,Works great as a guitar bench mat. Not rugged ...
2,1708,['https://images-na.ssl-images-amazon.com/imag...,We use these for everything from our acoustic ...
3,2078,['https://images-na.ssl-images-amazon.com/imag...,Great price and good quality. It didn't quite...
4,801,['https://images-na.ssl-images-amazon.com/imag...,I bought this bass to split time as my primary...
...,...,...,...
995,1265,['https://images-na.ssl-images-amazon.com/imag...,Extremely impressed with this kit.
996,1882,['https://images-na.ssl-images-amazon.com/imag...,This is a great stereo reverb with plenty of c...
997,1547,['https://images-na.ssl-images-amazon.com/imag...,I really like the simplicity of this bridge. I...
998,1004,['https://images-na.ssl-images-amazon.com/imag...,"Great Product, but there is no warranty in the..."


In [73]:
data.columns


Index(['Unnamed: 0', 'Image', 'Review Text'], dtype='object')

In [74]:
import matplotlib.pyplot as plt
import numpy as np
import requests
from PIL import Image
from io import BytesIO

url2data = {}
id2data = {}
urlList = []
review2id = {}
url2features = {}
removedUrl=[]

for image in data.index:
  for url in eval(data['Image'][image]):
    try:
      response = requests.get(url)
      retImage=Image.open(BytesIO(response.content))
      urlList.append(url)
      url2data[url]={'image':retImage,'id':data['Unnamed: 0'][image],'review':data['Review Text'][image]}


    except Exception as e:
      removedUrl.append(url)

  id2data[data['Unnamed: 0'][image]]={'review':data['Review Text'][image],'url':eval(data['Image'][image])}
  review2id[data['Review Text'][image]]=data['Unnamed: 0'][image]


In [75]:
print(len(urlList),len(removedUrl))

1640 8


In [76]:
import tensorflow as tf
from PIL import Image
from PIL import ImageEnhance
from PIL import ImageOps
import random

def image_feature_extraction_model(input_shape=(88, 88, 3)):
    model = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
    model.trainable = False
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    feature_extraction_layer = global_average_layer(model.output)
    feature_extraction_model = tf.keras.Model(inputs=model.input, outputs=feature_extraction_layer)

    return feature_extraction_model


def preprocess_image(image_array, target_size=(88, 88)):
    img = Image.fromarray(np.array(image_array))
    img = img.resize(target_size)
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(1.3)

    enhancer = ImageEnhance.Brightness(img)
    img = enhancer.enhance(1.3)

    if np.random.rand()>0.5:
      img = ImageOps.mirror(img)

    if np.random.rand()>0.5:
      img = ImageOps.flip(img)

    return np.expand_dims(np.array(img),axis=0)

model = image_feature_extraction_model()

In [77]:
for url in urlList:
  url2features[url]=model(preprocess_image(url2data[url]['image']))

In [78]:
import pickle

with open('/content/drive/MyDrive/CSE508_Winter2024_A2_2020559/u2f.pkl','wb') as f:
  pickle.dump(url2features,f)

with open('/content/drive/MyDrive/CSE508_Winter2024_A2_2020559/u2f.pkl','rb') as f:
  url2features = pickle.load(f)


In [81]:
import math
from collections import Counter
from nltk.stem import PorterStemmer, WordNetLemmatizer
porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')

def calculate_tf(document):
    word_counts = Counter(document)
    total_words = len(document)
    tf = {word: count / total_words for word, count in word_counts.items()}
    return tf

def calculate_idf(corpus, term):
    num_documents_with_term = sum(1 for doc in corpus if term in doc)
    num_documents = len(corpus)
    idf = math.log(num_documents / (1 + num_documents_with_term))
    return idf

def calculate_tfidf(document, corpus):
    tfidf = {}
    tf = calculate_tf(document)

    for term in tf:
        idf = calculate_idf(corpus, term)
        tfidf[term] = tf[term] * idf

    return tfidf

reviews = [review for review in data['Review Text']]

review2features={}

def preprocess(text,prnt=False):
    text = str(text).lower()

    tokens = word_tokenize(text)

    filtered_tokens = [word for word in tokens if word not in stopwords.words('english')]

    filtered_tokens = [word for word in filtered_tokens if word not in string.punctuation]

    filtered_tokens = [word for word in filtered_tokens if word.strip()]

    filtered_tokens = [porter_stemmer.stem(word) for word in filtered_tokens]

    filtered_tokens = [wordnet_lemmatizer.lemmatize(word) for word in filtered_tokens]

    return filtered_tokens


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [82]:
tokenized_documents = [preprocess(i) for i in reviews]
for doc in (reviews):
    review2features[doc] = calculate_tfidf(preprocess(doc), tokenized_documents)

In [83]:
import pickle

with open('/content/drive/MyDrive/CSE508_Winter2024_A2_2020559/r2f.pkl','wb') as f:
  pickle.dump(review2features,f)

with open('/content/drive/MyDrive/CSE508_Winter2024_A2_2020559/r2f.pkl','rb') as f:
  review2features = pickle.load(f)


In [84]:
import math

def calculate_cosine_similarity(tfidf_doc1, tfidf_doc2):
    dot_product = sum(tfidf_doc1.get(term, 0) * tfidf_doc2.get(term, 0) for term in set(tfidf_doc1) & set(tfidf_doc2))

    magnitude_doc1 = math.sqrt(sum(score**2 for score in tfidf_doc1.values()))
    magnitude_doc2 = math.sqrt(sum(score**2 for score in tfidf_doc2.values()))

    cosine_similarity = dot_product / (magnitude_doc1 * magnitude_doc2) if magnitude_doc1 * magnitude_doc2 != 0 else 0

    return cosine_similarity


#Image Retrieval

In [85]:
from sklearn.metrics.pairwise import cosine_similarity
inp_url = input("Input url ")
inp_review = input("Input Review ")
cosine_scores = [[cosine_similarity(url2features[inp_url],url2features[tempUrl])[0][0],tempUrl] for tempUrl in urlList if url2data[tempUrl]['id']!=url2data[inp_url]['id']]
cosine_scores.sort(key=lambda l:l[0],reverse=True)
print('\n\n')
for indexx in range(3):
  tempUrls = cosine_scores[indexx][1]
  tempReviews = url2data[tempUrls]['review']
  tempIscore = cosine_scores[indexx][0]
  tempRscore = calculate_cosine_similarity(review2features[tempReviews],review2features[inp_review])

  print('Image URL: ',tempUrls)
  print("Review: ",tempReviews)
  print("Cosine similarity of images: ", tempIscore)
  print("Cosine similarity of reviews: ", tempRscore)
  print('Composite similarity score: ', (tempIscore+tempRscore)/2,'\n')

Input url https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg
Input Review Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.



Image URL:  https://images-na.ssl-images-amazon.com/images/I/8128C2ja3vL._SY88.jpg
Review:  IMO this is the best and only way to mount toms, especially Pearls.  My Session kit came with them and with the heavy 6 play maple toms have never had a single issue with tuning, bouncing, lug stress whatsoever.  I think after you use them for a while you can take them for granted and don't fully understand how sturdy they are until you use another suspension system.  I recently purchased a Pearl Birch Vision kit for my wife which came with Pearl's I.S.S. mounting system.  To be fair I've never used them, and in the case of the new kit they came right off, and the Optimounts went right on.  Even though these

#Review Retrieval

In [89]:
inp_url = input("Input url ")
inp_review = input("Input Review ")
cosine_scores = [[calculate_cosine_similarity(review2features[inp_review],review2features[tempReview]),tempReview] for tempReview in data['Review Text'] if tempReview!=inp_review and isinstance(tempReview, str)]
cosine_scores.sort(key=lambda l:l[0],reverse=True)
print('\n\n')
for indexx in range(3):
  tempUrls = id2data[review2id[cosine_scores[indexx][1]]]['url']
  tempReviews = cosine_scores[indexx][1]
  tempIscore = np.mean([cosine_similarity(url2features[inp_url],url2features[tempUrl]) for tempUrl in tempUrls])
  if len(tempUrls)==0:
    tempIscore=0
  tempRscore = cosine_scores[indexx][0]

  print('Image URL: ',tempUrls)
  print("Review: ",tempReviews)
  print("Cosine similarity of images: ", tempIscore)
  print("Cosine similarity of reviews: ", tempRscore)
  print('Composite similarity score: ', (tempIscore+tempRscore)/2,'\n')


Input url https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg
Input Review Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.



Image URL:  ['https://images-na.ssl-images-amazon.com/images/I/81Z1d7HaBfL._SY88.jpg']
Review:  Nice solid springs and defeinitely more silent. Easy installation and the black looks cool.

Pictured with some old uninstalled springs next to them.
Cosine similarity of images:  0.62275696
Cosine similarity of reviews:  0.3331900140257178
Composite similarity score:  0.47797348601676515 

Image URL:  ['https://images-na.ssl-images-amazon.com/images/I/61rFj+KtBRL._SY88.jpg']
Review:  Fits great but the only complaint I have is that the tremolo springs are cheap quality and so is the block that attaches to the bridge. I took it off and put my own on. When you do that though, the wammy bar won't fit.
Cosi

# Composite Score

In [91]:
inp_url = input("Input url ")
inp_review = input("Input Review ")
cosine_scores = [[(cosine_similarity(url2features[inp_url],url2features[tempUrl])[0][0]+calculate_cosine_similarity(review2features[inp_review],review2features[url2data[tempUrl]['review']]))/2,tempUrl] for tempUrl in urlList if url2data[tempUrl]['id']!=url2data[inp_url]['id'] and isinstance(url2data[tempUrl]['review'],str)]
cosine_scores.sort(key=lambda l:l[0],reverse=True)
print('\n\n')
for indexx in range(3):
  tempUrls = cosine_scores[indexx][1]
  tempReviews = url2data[tempUrls]['review']
  tempIscore = cosine_similarity(url2features[inp_url],url2features[tempUrls])[0][0]
  tempRscore = calculate_cosine_similarity(review2features[tempReviews],review2features[inp_review])

  print('Image URL: ',tempUrls)
  print("Review: ",tempReviews)
  print("Cosine similarity of images: ", tempIscore)
  print("Cosine similarity of reviews: ", tempRscore)
  print('Composite similarity score: ', cosine_scores[indexx][0],'\n')

Input url https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg
Input Review Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.



Image URL:  https://images-na.ssl-images-amazon.com/images/I/81U3GJsTjNL._SY88.jpg
Review:  Great Quality, adjustable tension. Well made.
Cosine similarity of images:  0.8492337
Cosine similarity of reviews:  0.2172338398715884
Composite similarity score:  0.5332337633977846 

Image URL:  https://images-na.ssl-images-amazon.com/images/I/71rpRdEgKoL._SY88.jpg
Review:  Great pups!

Fender really got these right!
Great vintage tone pups at a steal.
Replaced original Lace pup equipped pickguard in my Strat Plus. And wired up a new pickguard with a .022 cap. Which is working fine for me.

If you also need pickup adjustment screws and you have a modern pick guard. Be aware these come with oval head count